#+TITLE: Optimal Brain Damage
#+CATEGORIES: Compression
#+DESCRIPTION: ...
#+DATE: 2024-01-29

# Optimal Brain Damage

Optimal Brain Damage is an approach to identify and remove unimportant weights from a neural network. The procedure involves training the network, computing the second derivatives for each parameter, computing the saliencies, sorting the parameters by saliency, and deleting some low-saliency parameters.

The saliency of a parameter is defined as the change in the objective function caused by deleting that parameter. Using the second derivative of the objective function with respect to the parameters avoids the prohibitive labor of directly evaluating the saliency by temporarily deleting each parameter and reevaluating the objective function​​.

Second order methods are typically difficult with neural networks due to the enormity of the Hessian matrix. Optimal Brain Damage introduces a simple diagonal approximation where the change in the objective function $E$ caused by deleting several parameters is the sum of the changes caused by deleting each parameter individually.

The OBD method introduces simplifying approximations due to the practical insolubility of finding a set of parameters whose deletion will cause the least increase of the objective function, E. The difficulty arises from the enormity of the Hessian matrix, H, and its computational challenges. The "diagonal" approximation is used where the change in E caused by deleting several parameters is the sum of the changes caused by deleting each parameter individually​​.

The method uses a "quadratic" approximation, assuming the cost function is nearly quadratic, allowing the neglect of the last term in the equation, simplifying the change in the objective function, ΔEΔE, to ΔE=12∑ihii(Δui)2ΔE=21​∑i​hii​(Δui​)2​​.

The second derivatives, crucial for the method, are computed efficiently. The diagonal terms of the Hessian, hiihii​, are given by ∑(i,j)∈Vk∂wij∂uk∂2E∂wij2∑(i,j)∈Vk​​∂uk​∂wij​​∂wij2​∂2E​, where the summand can be expanded as ∂2E∂wij2=zi∂2E∂ai2∂wij2​∂2E​=zi​∂ai2​∂2E​. These second derivatives are back-propagated from layer to layer​

The OBD procedure involves training the network, computing the second derivatives for each parameter, computing the saliencies (Sk=huuu2/2Sk​=huu​u2/2), sorting the parameters by saliency, and deleting some low-saliency parameters. This process can be iterated, with several variants possible, such as decreasing the values of low-saliency parameters instead of setting them to zero or allowing deleted parameters to adapt again after being set to zero​
​.